# 03 - Model Training
Train model and log to MLflow, register model.

In [ ]:
import dagshub
dagshub.init(repo_owner='yahiaehab10', repo_name='MLflow_demo_MF', mlflow=True)
from src.train import train_model
from src.evaluate import evaluate_model

# Train and evaluate
train_model()
# Replace with actual run_id printed above
run_id = 'YOUR_RUN_ID'
evaluate_model(run_id)